In [1]:
"""
NICOLE JOSEPH
Deep Learning HW 4
Classify CIFAR10
"""

'\nNICOLE JOSEPH\nDeep Learning HW 4\nClassify CIFAR10\n'

In [2]:
import gzip
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import Tensor
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Add, Flatten, Dropout, ReLU, BatchNormalization, Input, Dense, ZeroPadding2D, AveragePooling2D
from keras.models import Model

In [23]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# python routine to open file and return a dictionary
# citation: https://www.cs.toronto.edu/~kriz/cifar.html 
def unpickle(file_paths):
    data = []
    labels = []
    for i, file_path in enumerate(file_paths):
        print("processing " + file_path)
        with open(file_path, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        # Each data_batch file contains a dictionary with the elements data and labels
        # In the dictionary, data is a 10,000x3072 numpy array of uint8s     
        data = np.append(data, dict[b'data']).astype(np.uint8)
        data = data.reshape(10000*(i+1), 3, 32, 32)
        # In the dictionary, labels is a list of 10,000 numbers in the range 0-9
        labels = np.append(labels, dict[b'labels']).astype(np.uint8)
    data = data.transpose(0,2,3,1)
    return data, labels

In [6]:
training_dataset = ["drive/MyDrive/deep-learning/cifar10/data_batch_1", "drive/MyDrive/deep-learning/cifar10/data_batch_2", "drive/MyDrive/deep-learning/cifar10/data_batch_3",
                  "drive/MyDrive/deep-learning/cifar10/data_batch_4", "drive/MyDrive/deep-learning/cifar10/data_batch_5"]
test_dataset = ["drive/MyDrive/deep-learning/cifar10/test_batch"]

x_train, y_train = unpickle(training_dataset)
x_test, y_test = unpickle(test_dataset)

processing drive/MyDrive/deep-learning/cifar10/data_batch_1
processing drive/MyDrive/deep-learning/cifar10/data_batch_2
processing drive/MyDrive/deep-learning/cifar10/data_batch_3
processing drive/MyDrive/deep-learning/cifar10/data_batch_4
processing drive/MyDrive/deep-learning/cifar10/data_batch_5
processing drive/MyDrive/deep-learning/cifar10/test_batch


In [7]:
# Using Keras Functional API to construct a Residual Neural Network
# citation: https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba 

In [8]:
# helper function that takes a tensor as input and adds relu and batch normalization to it
# citation: https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba 
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

In [9]:
# construct a residual block
# citation: https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba 
def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

In [10]:
# citation: https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba 
def create_res_net():
    inputs = Input(shape=(32, 32, 3))
    num_filters = 64
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    num_blocks_list = [2, 2, 2, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(10, activation='softmax')(t)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [11]:
model = create_res_net()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 3)   12          ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   1792        ['batch_normalization[0][0]']    
                                                                                                  
 re_lu (ReLU)                   (None, 32, 32, 64)   0           ['conv2d[0][0]']             

In [19]:
# history = model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2, shuffle = True, steps_per_epoch=len(x_train) // 128)
#history = model.fit(x_train, y_train, steps_per_epoch=len(x_train) // 128, epochs=10, verbose=1, validation_split=0.2, shuffle = True,  validation_steps = len(x_test)//128)
history = model.fit(x_train, y_train, batch_size=128, epochs=15, verbose=1, validation_split=0.2, shuffle = True)
print ("All done!")

Epoch 1/15
313/313 [==============================] - 36s 116ms/step - loss: 0.0134 - accuracy: 0.9955 - val_loss: 0.9601 - val_accuracy: 0.8400
Epoch 2/15
313/313 [==============================] - 36s 115ms/step - loss: 0.0097 - accuracy: 0.9966 - val_loss: 1.0043 - val_accuracy: 0.8357
Epoch 3/15
313/313 [==============================] - 36s 115ms/step - loss: 0.0140 - accuracy: 0.9955 - val_loss: 0.9730 - val_accuracy: 0.8351
Epoch 4/15
313/313 [==============================] - 36s 115ms/step - loss: 0.0108 - accuracy: 0.9963 - val_loss: 0.9430 - val_accuracy: 0.8437
Epoch 5/15
313/313 [==============================] - 36s 115ms/step - loss: 0.0084 - accuracy: 0.9974 - val_loss: 0.9720 - val_accuracy: 0.8428
Epoch 6/15
313/313 [==============================] - 36s 116ms/step - loss: 0.0120 - accuracy: 0.9961 - val_loss: 0.9100 - val_accuracy: 0.8480
Epoch 7/15
313/313 [==============================] - 36s 115ms/step - loss: 0.0161 - accuracy: 0.9945 - val_loss: 0.9492 - val_ac

In [20]:
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 4s 12ms/step - loss: 1.0200 - accuracy: 0.8342
Test loss: 1.0200449228286743
Test accuracy: 0.8342000246047974
